In [21]:
from skfuzzy import control as ctrl
import pickle
import pandas as pd
import random

In [22]:
def diagnose_mental_health(diagnosis):
    if diagnosis <= 33:
        return "Stable"
    elif diagnosis <= 66:
        return "Moderate"
    else:
        return "Severe"

In [23]:
with open("mental_health_diagnosis.pickle", "rb") as f:
    mental_health_ctrl = pickle.load(f)

In [24]:
def run_fuzzy_system(behavior, speech, medical_history):
    mental_health_diagnosis = ctrl.ControlSystemSimulation(mental_health_ctrl)
    mental_health_diagnosis.input["Behavior"] = behavior
    mental_health_diagnosis.input["Speech"] = speech
    mental_health_diagnosis.input["Medical History"] = medical_history
    mental_health_diagnosis.compute()
    crisp_output = mental_health_diagnosis.output["Mental Health Condition"]

    mental_health = diagnose_mental_health(crisp_output)

    return mental_health

In [25]:
df = pd.read_excel("test_data.xlsx")

In [26]:
df

,Behavior,Speech,Medical History,Expected Mental Health Condition
0,Normal,Coherent,Minimal,Stable
1,Abnormal,Disorganized,Significant,Severe
2,Moderate,Mildly Disorganized,Moderate,Moderate
3,Normal,Mildly Disorganized,Significant,Severe
4,Abnormal,Coherent,Minimal,Severe
5,Moderate,Coherent,Significant,Severe
6,Normal,Coherent,Moderate,Moderate
7,Abnormal,Disorganized,Significant,Severe
8,Moderate,Mildly Disorganized,Minimal,Stable
9,Normal,Coherent,Significant,Moderate


In [27]:
df["Behavior"] = df["Behavior"].map(
        {
            "Normal": random.uniform(0, 10),
            "Moderate": random.uniform(40, 60),
            "Abnormal": random.uniform(90, 100),
        }
    )

df["Speech"] = df["Speech"].map(
    {
        "Coherent": random.uniform(0, 10),
        "Mildly Disorganized": random.uniform(40, 60),
        "Disorganized": random.uniform(90, 100),
    }
)

df["Medical History"] = df["Medical History"].map(
    {
        "Minimal": random.uniform(0, 10),
        "Moderate": random.uniform(40, 60),
        "Significant": random.uniform(90, 100),
    }
)

In [28]:
df.head()

,Behavior,Speech,Medical History,Expected Mental Health Condition
0,2.374907,8.943787,0.354923,Stable
1,97.212148,99.831369,94.237469,Severe
2,40.886510,56.046413,45.264601,Moderate
3,2.374907,56.046413,94.237469,Severe
4,97.212148,8.943787,0.354923,Severe


In [29]:
def predict(row):
    return run_fuzzy_system(row["Behavior"], row["Speech"], row["Medical History"])

df["Predicted Mental Health Condition"] = df.apply(predict, axis=1)

In [30]:
df

,Behavior,Speech,Medical History,Expected Mental Health Condition,Predicted Mental Health Condition
0,2.374907,8.943787,0.354923,Stable,Stable
1,97.212148,99.831369,94.237469,Severe,Severe
2,40.886510,56.046413,45.264601,Moderate,Moderate
3,2.374907,56.046413,94.237469,Severe,Severe
4,97.212148,8.943787,0.354923,Severe,Severe
5,40.886510,8.943787,94.237469,Severe,Severe
6,2.374907,8.943787,45.264601,Moderate,Stable
7,97.212148,99.831369,94.237469,Severe,Severe
8,40.886510,56.046413,0.354923,Stable,Moderate
9,2.374907,8.943787,94.237469,Moderate,Stable


In [31]:
correct_predictions = (
    df["Predicted Mental Health Condition"]
    == df["Expected Mental Health Condition"]
).sum()

accuracy = correct_predictions / len(df)

print(f"Accuracy: {accuracy}")

Accuracy: 0.7


In [32]:
TP = ((df["Predicted Mental Health Condition"] == "Severe") & (df["Expected Mental Health Condition"] == "Severe")).sum()
FP = ((df["Predicted Mental Health Condition"] == "Severe") & (df["Expected Mental Health Condition"] != "Severe")).sum()
FN = ((df["Predicted Mental Health Condition"] != "Severe") & (df["Expected Mental Health Condition"] == "Severe")).sum()

# Calculate the precision for the "Severe" class
precision = TP / (TP + FP)

print(f"Precision (serve class): {precision}")

Precision (serve class): 1.0
